In [1]:
#pip install torch

In [24]:
import torch
import torch.nn as nn

import numpy as np

## Задача №1

In [3]:
x = torch.tensor([1, 2, 3, 4, 5]).float()
y_grad = torch.tensor([2.5]).float()

In [4]:
class MyLayer(nn.Module):
    def __init__(self, x):
        super(MyLayer, self).__init__()
        self.x = x
        self.linear1 = nn.Linear(5, 5, bias=True)
        self.linear2 = nn.Linear(5, 1, bias=True)

    def forward(self):
        z = self.linear1(self.x)
        g = nn.Tanh()
        y_pred = self.linear2(g(z))
        return y_pred

    def backward(self, grad_y_pred):
        grad_w2 = self.x.view(5, 1) * y_grad
        g_der = nn.Tanh()
        grad_g = 1 - g_der(grad_w2).pow(2)
        grad_w1 =  self.x.view(5, 1) * grad_g.t()
        return grad_w1, grad_w2

In [5]:
layer = MyLayer(x)
y = layer.forward()
print(y)

tensor([-0.2840], grad_fn=<AddBackward0>)


In [6]:
grad_w1, grad_w2 = layer.backward(y_grad)
print(grad_w2)
print(grad_w1)

tensor([[ 2.5000],
        [ 5.0000],
        [ 7.5000],
        [10.0000],
        [12.5000]])
tensor([[2.6592e-02, 1.8156e-04, 1.1921e-06, 0.0000e+00, 0.0000e+00],
        [5.3185e-02, 3.6311e-04, 2.3842e-06, 0.0000e+00, 0.0000e+00],
        [7.9777e-02, 5.4467e-04, 3.5763e-06, 0.0000e+00, 0.0000e+00],
        [1.0637e-01, 7.2622e-04, 4.7684e-06, 0.0000e+00, 0.0000e+00],
        [1.3296e-01, 9.0778e-04, 5.9605e-06, 0.0000e+00, 0.0000e+00]])


## Задача №2

In [7]:
class SGDMomentum:
    def __init__(self, model_weights, momentum: float=0.99, lr: float = 0.001):
        self.momentum = momentum
        self.lr = lr
        self.velocity = torch.zeros_like(model_weights)
        self.weights = model_weights

    def step(self, grad):
        self.velocity = self.momentum * self.velocity - self.lr * grad
        self.weights = self.weights + self.velocity

In [53]:
class RMSprop:
    def __init__(self, model_weights, rho: float=0.99, lr: float = 0.01):
        self.rho = rho
        self.lr = lr
        self.accumulated = torch.zeros_like(model_weights)
        self.weights = model_weights

    def step(self, grad):
        self.accumulated = self.rho * self.accumulated + (1 - self.rho) * (grad ** 2)
        self.weights = self.weights - self.lr / np.sqrt(self.accumulated + 10**(-10)) * grad

## Задача №3

In [110]:
def func(x):
    #return (2*x-7)**2-4
    #return x**2 - 2*x + 1
    return x**2 - 10*x + 9
    
def grad(x):
    #return 4*(2*x-7)
    #return 2*x - 2
    return 2*x - 10

def min_value(init_x):
    x = torch.tensor(init_x)
    g = grad(x)
    optim = RMSprop(x, 0.9, 0.01)
    for i in range(1000):
        optim.step(g)
        g = grad(optim.weights)
    print('min = ', optim.weights)
    return optim.weights

def solver(init_x):
    
    min_val = min_value(init_x)
    
    error = 10**(-8)
    f = func(init_x)
    x = torch.tensor(init_x)
    g = grad(x)
    optim = RMSprop(x, 0.9, 0.001)
    while f > error:
        optim.step(g)
        g = grad(optim.weights)
        f = func(optim.weights)
    
    root_1 = optim.weights
    root_2 = 2 * min_val - root_1
    
    print('root_1 = ', optim.weights) 
    print('root_2 = ', root_2) 

In [111]:
min_value(3)

min =  tensor(5.)


tensor(5.)

In [112]:
solver(10)

min =  tensor(5.)
root_1 =  tensor(8.9999)
root_2 =  tensor(1.0001)
